In [1]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed, BatchNormalization, Dropout, Reshape
from keras.optimizers import Adam
from keras.models import Sequential, load_model, Model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import time
import pandas as pd 

2024-08-11 15:36:18.333854: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 15:36:18.403397: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 15:36:18.423403: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-11 15:36:18.540095: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-11 15:36:20.457605: W tensorflow/compiler/tf2

In [2]:
class CLM():
    def __init__(self, layers, n_chars, dropouts, trainables,lr):
        self.layers = layers
        self.n_chars = n_chars
        self.dropouts = dropouts
        self.trainables = trainables
        self.lr = lr
        self.model = None
        self.build_model()
    def build_model(self):
        # create a squential model and assigned to self.model
        self.model = Sequential()
        # Add a BatchNormalization layer input layer, input shape should be None, number of characters
        self.model.add(BatchNormalization(input_shape = (None,self.n_chars)))
        
        # Add n LSTM layers, where n is defined by the variable layers. This will be a list of ints where each int is the number of units in the LSTM layer. i.e [256, 512] will add two layers with 256 and 512 neurons respectively
        # This loop should also include the dropout values contained in the dropouts variable (list of floats). 
        # And it should also include the trainable parameter from the trainables variable (list of booleans). 
        print(self.layers)
        for i in range(len(self.layers)):
            print(self.layers[i])
            self.model.add(LSTM(units=self.layers[i], trainable=self.trainables[i], return_sequences=True, dropout=self.dropouts[i]))
            
        # add another BatchNormalization layer, this time no need to specify input shape
        self.model.add(BatchNormalization())
        # Finally add an output layer, it should be a Dense layer with n_chars units and softmax activation. Also, this layer should be containd in a TimeDistributed layer, so that it can be applied to each character in the sequence.
        self.model.add(TimeDistributed(Dense(self.n_chars, activation='softmax')))
        # compile the model with Adam optimizer and categorical_crossentropy loss. Don't forget to set the learning rate to the value contained in the lr variable with optimizer = Adam(learning_rate=self.lr)
        optimizer = Adam(learning_rate=self.lr)
        self.model.compile(optimizer=optimizer, loss='categorical_crossentropy')

In [3]:
#Create a checkpointer callback that saves the model weights to a file every epoch. The filename should be '{epoch:03d}.weights.h5'
checkpointer = ModelCheckpoint(
    filepath='{epoch:03d}.weights.h5',
    save_weights_only=True,
    save_freq='epoch'
)

In [4]:
import re
import numpy as np
import keras

#Fixed parameters
PROCESSING_FIXED = {'start_char': 'G', 
                    'end_char': 'E', 
                    'pad_char': 'A'}

INDICES_TOKEN = {0: 'c',
                 1: 'C',
                 2: '(',
                 3: ')',
                 4: 'O',
                 5: '1',
                 6: '2',
                 7: '=',
                 8: 'N',
                 9: '@',
                 10: '[',
                 11: ']',
                 12: 'n',
                 13: '3',
                 14: 'H',
                 15: 'F',
                 16: '4',
                 17: '-',
                 18: 'S',
                 19: 'Cl',
                 20: '/',
                 21: 's',
                 22: 'o',
                 23: '5',
                 24: '+',
                 25: '#',
                 26: '\\',
                 27: 'Br',
                 28: 'P',
                 29: '6',
                 30: 'I',
                 31: '7',
                 32: PROCESSING_FIXED['start_char'],
                 33: PROCESSING_FIXED['end_char'],
                 34: PROCESSING_FIXED['pad_char']}                
TOKEN_INDICES = {v: k for k, v in INDICES_TOKEN.items()}

def smi_tokenizer(smi):
    """
    Tokenize a SMILES
    """
    pattern =  "(\[|\]|Xe|Ba|Rb|Ra|Sr|Dy|Li|Kr|Bi|Mn|He|Am|Pu|Cm|Pm|Ne|Th|Ni|Pr|Fe|Lu|Pa|Fm|Tm|Tb|Er|Be|Al|Gd|Eu|te|As|Pt|Lr|Sm|Ca|La|Ti|Te|Ac|Si|Cf|Rf|Na|Cu|Au|Nd|Ag|Se|se|Zn|Mg|Br|Cl|U|V|K|C|B|H|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%\d{2}|\d)"
    regex = re.compile(pattern)
    tokens = [token for token in regex.findall(smi)]

    return tokens        

class onehotencoder():
    def __init__(self, max_chars=90):
        # here we define the class variables that will be used in the functions. If we want to use them in the class, we need to use self. in front of the variable name
        # then when we want to use the class variable in the function, we pass self as the first argument to the function and we can access the class variables
        self.max_chars = max_chars
    def smiles_to_num(self, smiles):
        tokens = smi_tokenizer(smiles)
        tokens = [PROCESSING_FIXED['start_char']] + tokens +[PROCESSING_FIXED['end_char']]
        tokens+= [PROCESSING_FIXED['pad_char']]* (self.max_chars-len(tokens))
        num_tokens = [TOKEN_INDICES[t] for t in tokens]
        #print(tokens)
        #create numbers
        return np.asarray(tokens) #Return the numbers as a numpy array
    def num_to_onehot(self,tokens):
        onehot = np.zeros((len(tokens), len(INDICES_TOKEN)))
        for i,token in enumerate(tokens):
            onehot[i, TOKEN_INDICES[token]] = 1
        return onehot
        #This function will convert the numbers to one hot encoding. It should take a list of numbers and return a list of one hot encoded vectors with the shape (max_chars, 35)
    def generate_data(self, smiles):
        nums = self.smiles_to_num(smiles)
        data = self.num_to_onehot(nums)
        return np.asarray(data)
        #This function should take a list of smiles strings and return a numpy array of one hot encoded vectors with the shape (number of smiles, max_chars, 35)
        #You can use the smiles_to_num and num_to_onehot functions to help you with this
        #This function will be used to generate the training and validation data

        
class DataGenerator(keras.utils.Sequence):
    def __init__(self, list_IDs, batch_size, data, max_len, num_chars, use_multiprocessing=True, workers=6, shuffle=True):
        super().__init__(use_multiprocessing=use_multiprocessing, workers=workers)
        'Initialization'
        self.encoder = onehotencoder(max_chars=max_len)
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.data = data
        self.num_chars = num_chars
        self.max_len = max_len
        self.shuffle = shuffle
        self.on_epoch_end()

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y
    
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.max_len-1, self.num_chars))
        y = np.empty((self.batch_size, self.max_len-1, self.num_chars))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            smiles = self.data[ID]
            one_hot_smi = self.encoder.generate_data(smiles)
            X[i] = one_hot_smi[:-1]
            y[i] = one_hot_smi[1:]
        return X, y
    def tester(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.max_len-1, self.num_chars))
        y = np.empty((self.batch_size, self.max_len-1, self.num_chars))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            smiles = self.data[ID]
            one_hot_smi = self.encoder.generate_data(smiles)
            X[i] = one_hot_smi[:-1]
            y[i] = one_hot_smi[1:]
        return X, y
    #You can give this a try with the tutorial and see if you can get it to work https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
    #This will be used to generate the data for the model. It should take a list of smiles strings and the corresponding labels and return the one hot encoded vectors in batches
    #You can use the onehotencoder class to help you with this.
    #It is important to follow the function signature of the keras.utils.Sequence class so that it can be used with the model.fit_generator function
    #This is al detailed in the tutorial above

In [5]:
#load the data from data/us_pharma_patent_data_lowe_smiles_can_unique_stereochem.txt
data = pd.read_csv('data/us_pharma_patent_data_lowe_smiles_can_unique_stereochem.txt', sep='\t', header=None)
#Remember to drop missing values and duplicates
data = data.dropna().drop_duplicates()
#Also, remove any smiles string that contains a character NOT in our vocabulary (excluding pad, start and end chars). Hint: allowed_chars = [t for t in TOKEN_INDICES.keys()][:-3]
allowed_chars = [t for t in TOKEN_INDICES.keys()][:-3]
data = data[data[0].apply(lambda x: all(char in allowed_chars for char in x))]
#drop data longer than 90 characters
data = data[data[0].apply(lambda x: len(x)<=90)]
#Split the data into train and test sets with a 80/20 split. Don't forget to reset the index of the dataframes before splitting, so then we can use the train.index and test.index to create the generators
data = data.reset_index(drop=True)
train_data, test_data = train_test_split(data, test_size=0.2)


In [6]:
#Initialize the generators, we need one for training and one for validation. Use batch size of 256 for both. Remember to pass all the data to them, but the corresponding indices. Also the data should be passed as a list of strings, so don't forget to use the .tolist() method on the dataframe column

#confused about variables
train_generator = DataGenerator(
    data = data[0],
    max_len = 92,
    batch_size=256,
    list_IDs=train_data.index,
    num_chars=35
)

validation_generator = DataGenerator(
    data = data[0],
    max_len = 92,
    batch_size=256,
    list_IDs=test_data.index,
    num_chars=35
)

In [7]:
#Here are some predefine hyperparameters (taken from the paper which are already optimize, lucky us!)
layers = [1024,256]
dropouts = [0.40, 0.40]
trainables = [True, True]
lr = 0.001
epochs = 2

In [8]:
#Create the model object from the CLM class
chem_model = CLM(
    layers=layers,
    dropouts=dropouts,
    trainables=trainables,
    lr=lr,
    n_chars=35
)
#not sure why I am getting dimension error?

/home/scro4068/.local/lib/python3.10/site-packages/keras/src/layers/normalization/batch_normalization.py:143: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1723386989.167231   32684 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-11 15:36:29.475714: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[1024, 256]
1024
256


In [9]:
chem_model.model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization             │ (None, None, 35)       │           140 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 1024)     │     4,341,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, None, 256)      │     1,311,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, None, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, None, 35)       │         8,995 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,663,663 (21.61 MB)

 Trainable params: 5,663,081 (21.60 MB)

 Non-trainable params: 582 (2.27 KB)

In [ ]:
#Do training! (pass the generators instead of the data directly) 
chem_model.model.fit(train_generator, validation_data=validation_generator, epochs=2, callbacks=[checkpointer])

Epoch 1/2
  11/1910 ━━━━━━━━━━━━━━━━━━━━ 1:48:14 3s/step - loss: 2.4107

Process Keras_worker_ForkPoolWorker-5:
Process Keras_worker_ForkPoolWorker-3:
Process Keras_worker_ForkPoolWorker-4:
Process Keras_worker_ForkPoolWorker-6:
Process Keras_worker_ForkPoolWorker-2:
Process Keras_worker_ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.

KeyboardInterrupt: 

Process Keras_worker_ForkPoolWorker-12:
Process Keras_worker_ForkPoolWorker-8:
Process Keras_worker_ForkPoolWorker-10:
Process Keras_worker_ForkPoolWorker-7:
